In [3]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression  # Usar el modelo Logistic Regression

# Cargar el conjunto completo de datos de entrenamiento y el de prueba
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')  # Cargar el conjunto completo de prueba

# Paso 1: Imputación de valores faltantes en columnas categóricas
categorical_columns = [
    'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA', 'FAMI_ESTRATOVIVIENDA',
    'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO'
]

for column in categorical_columns:
    # Rellenar valores faltantes en el conjunto de entrenamiento y prueba con la moda del conjunto de entrenamiento
    mode_value = train_data[column].mode()[0]
    train_data[column] = train_data[column].fillna(mode_value)
    if column in test_data.columns:
        test_data[column] = test_data[column].fillna(mode_value)

# Definir las características (X) y la variable objetivo (y) para el conjunto de entrenamiento
X_train = train_data.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'])
y_train = train_data['RENDIMIENTO_GLOBAL']

# Definir el conjunto de características (X_test) para el conjunto de prueba
X_test = test_data.drop(columns=['ID'])

# Identificar columnas numéricas y categóricas para el pipeline
numeric_features = ['PERIODO']
categorical_features = categorical_columns

# Crear transformadores para las columnas numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(drop='first', handle_unknown='ignore')

# Crear un preprocesador que aplique las transformaciones correspondientes
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear un pipeline que incluya el preprocesador y el modelo de Regresión Logística
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Entrenar el modelo en el conjunto completo de entrenamiento
pipeline.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba completo
all_predictions = pipeline.predict(X_test)

# Crear el DataFrame para el archivo de envío
submission = pd.DataFrame({
    'ID': test_data['ID'],  # Usar los IDs del conjunto de prueba completo
    'RENDIMIENTO_GLOBAL': all_predictions
})

# Guardar el archivo de envío en formato CSV
submission.to_csv('solucion.csv', index=False)
print("El archivo 'solucion.csv' ha sido generado y está listo para descargar.")

El archivo 'solucion.csv' ha sido generado y está listo para descargar.
